# Initialize

In [21]:
import os
from haystack.pipelines import Pipeline
from haystack.nodes import TextConverter
from haystack.nodes import PreProcessor
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import TransformersSummarizer
from transformers import GenerationConfig
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser
from haystack.nodes import EmbeddingRetriever

In [3]:
retriever_dict = dict()
text_converter_dict = dict() 
preprocessor_dict = dict()
document_store_dict = dict()
prompt_node_dict = dict()
hf_access_token = os.getenv('access_token_huggingface')

In [4]:
# https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads
embedding_models_list = [
    'sentence-transformers/all-mpnet-base-v2',
    'sentence-transformers/all-MiniLM-L6-v2',
    "sentence-transformers/multi-qa-mpnet-base-dot-v1"
]

# Indexing pipeline: Save and load document store

In [7]:
iteration = 1
text_converter = TextConverter() # https://docs.haystack.deepset.ai/docs/file_converters
text_converter_dict[iteration] = text_converter

preprocessor = PreProcessor( # https://docs.haystack.deepset.ai/docs/preprocessor
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
  	remove_substrings=None,
    split_by=None, # Unit for splitting the document. Can be "word", "sentence", or "passage". Set to None to disable splitting.
    split_length=200,
    split_respect_sentence_boundary=True,
    split_overlap=0,
  	max_chars_check = 100000
)
preprocessor_dict[iteration] = preprocessor

In [12]:
from datetime import datetime

def append_timestamp(string, ext=None):
    timestamp = datetime.now().strftime("%Y-%m-%d_%H%M")
    if ext:
        return f'{string}_{timestamp}.{ext}'
    else:
        return f'{string}_{timestamp}' 

index_filename = append_timestamp('journal_article_index')
config_filename = append_timestamp('journal_article_config')
faiss_filename = append_timestamp('faiss_document_store')
path = '../data/testing_2023-11-23/'
document_store = FAISSDocumentStore( # https://docs.haystack.deepset.ai/reference/document-store-api#faissdocumentstore
    sql_url=f"sqlite:///{path}{faiss_filename}.db",
    faiss_index_factory_str="Flat"
    )
document_store_dict[iteration] = document_store

In [9]:
retriever = EmbeddingRetriever( # https://docs.haystack.deepset.ai/reference/retriever-api
    document_store=document_store,
    embedding_model=embedding_models_list[iteration - 1],
    model_format="sentence_transformers",
    top_k = 100
)
retriever_dict[iteration] = retriever

/home/silvhua/solarathon/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [13]:

p = Pipeline()
p.add_node(component=text_converter, name="TextConverter", inputs=["File"])
p.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
p.add_node(component=document_store, name="DocumentStore", inputs=["PreProcessor"])
p.add_node(component=retriever, name="Retriever", inputs=["DocumentStore"])


p.run(file_paths=[f"{path}/journal_article.txt"])

Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 3350.08docs/s]
Writing Documents: 10000it [00:00, 673048.56it/s]       


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'documents': [<Document: {'content': 'High doses of anti-inflammatory drugs compromise muscle strength and hypertrophic adaptations to resistance training in young adults.\n\nDiscussion\nThe interest in this study was spurred by the intriguing observation that while NSAIDs might have a negative impact on acute exercise responses (satellite cell activity, translational signalling and protein synthesis), previous human studies have failed to demonstrate a detrimental effect of NSAIDs on the development of muscle hypertrophy in response to chronic resistance training in young adults, possibly due to differences in drug dosage across studies. Accordingly, in the current study, healthy young men and women performed 8 weeks of supervised resistance training with concomitant high- or low-dose NSAID treatment. The major and novel findings were that 1) maximal overthe-counter doses of ibuprofen compromised resistance exercise-induced muscle hypertropy\nindependent of training mode; 2) increase

## Update embeddings before saving

In [14]:
document_store.update_embeddings(retriever)
document_store.save(index_path=f'{path}/{index_filename}', config_path=f'{path}/{config_filename}')

Updating Embedding:   0%|          | 0/1 [00:00<?, ? docs/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Documents Processed: 10000 docs [00:01, 8612.66 docs/s]      


# Summarization pipeline

In [12]:
import os
from haystack.pipelines import Pipeline
from haystack.nodes import TextConverter
from haystack.nodes import PreProcessor
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import TransformersSummarizer
from transformers import GenerationConfig
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser
from haystack.nodes import EmbeddingRetriever

In [15]:
prompt_node_dict = dict()
output_dict = dict()
system_message_dict = dict()
prompt_dict = dict()

## models list

In [34]:
models_list = [
 'sshleifer/distilbart-cnn-12-6',
 'facebook/bart-large-cnn', 
 'philschmid/bart-large-cnn-samsum',
 'philschmid/flan-t5-base-samsum'
]

In [17]:
file_timestamp = '_2023-11-23_1341'
index_filename = 'journal_article_index'+file_timestamp
config_filename = 'journal_article_config'+file_timestamp
faiss_filename = 'faiss_document_store'+file_timestamp
# path = '../data/testing_2023-11-23/'
path = path
saved_document_store = FAISSDocumentStore.load(
    index_path=f'{path}/{index_filename}', config_path=f'{path}/{config_filename}'
    )
print(type(saved_document_store))
saved_document_store.get_all_documents()

<class 'haystack.document_stores.faiss.FAISSDocumentStore'>


[<Document: {'content': 'High doses of anti-inflammatory drugs compromise muscle strength and hypertrophic adaptations to resistance training in young adults.\n\nDiscussion\nThe interest in this study was spurred by the intriguing observation that while NSAIDs might have a negative impact on acute exercise responses (satellite cell activity, translational signalling and protein synthesis), previous human studies have failed to demonstrate a detrimental effect of NSAIDs on the development of muscle hypertrophy in response to chronic resistance training in young adults, possibly due to differences in drug dosage across studies. Accordingly, in the current study, healthy young men and women performed 8 weeks of supervised resistance training with concomitant high- or low-dose NSAID treatment. The major and novel findings were that 1) maximal overthe-counter doses of ibuprofen compromised resistance exercise-induced muscle hypertropy\nindependent of training mode; 2) increases in muscle st

In [18]:
len(saved_document_store.get_all_documents())

1

## Iteration 1 `sshleifer/distilbart-cnn-12-6`

In [22]:
iteration = iteration
model_name = models_list[iteration - 1]
max_length = 3000
system_message = """
Summarize the messages to create a Frequently Asked Questions document.
"""
prompt = PromptTemplate( # https://docs.haystack.deepset.ai/docs/prompt_node#prompttemplates
    prompt='{query}\n\n Messages: {join(documents)} \n\nSummary: '
)
prompt_dict[iteration] = prompt

prompt_node = PromptNode( # https://docs.haystack.deepset.ai/reference/prompt-node-api # https://docs.haystack.deepset.ai/docs/prompt_node#in-a-pipeline
    model_name, api_key=hf_access_token, 
    max_length=max_length, # The maximum number of tokens the generated text output can have,
    default_prompt_template=prompt,
    )
prompt_node_dict[iteration] = prompt_node
system_message_dict[iteration] = system_message


ValueError: Model sshleifer/distilbart-cnn-12-6 is not supported - no matching invocation layer found. Currently supported invocation layers are: [<class 'haystack.nodes.prompt.invocation_layer.open_ai.OpenAIInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.azure_open_ai.AzureOpenAIInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.chatgpt.ChatGPTInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.azure_chatgpt.AzureChatGPTInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.anthropic_claude.AnthropicClaudeInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.cohere.CohereInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.hugging_face.HFLocalInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.hugging_face_inference.HFInferenceEndpointInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.sagemaker_base.SageMakerBaseInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_lay

### facebook/bart-large-cnn

In [23]:
iteration = iteration
model_name = models_list[iteration]
max_length = 3000
system_message = """
Summarize the messages to create a Frequently Asked Questions document.
"""
prompt = PromptTemplate( # https://docs.haystack.deepset.ai/docs/prompt_node#prompttemplates
    prompt='{query}\n\n Messages: {join(documents)} \n\nSummary: '
)
prompt_dict[iteration] = prompt

prompt_node = PromptNode( # https://docs.haystack.deepset.ai/reference/prompt-node-api # https://docs.haystack.deepset.ai/docs/prompt_node#in-a-pipeline
    model_name, api_key=hf_access_token, 
    max_length=max_length, # The maximum number of tokens the generated text output can have,
    default_prompt_template=prompt,
    )
prompt_node_dict[iteration] = prompt_node
system_message_dict[iteration] = system_message


ValueError: Model facebook/bart-large-cnn is not supported - no matching invocation layer found. Currently supported invocation layers are: [<class 'haystack.nodes.prompt.invocation_layer.open_ai.OpenAIInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.azure_open_ai.AzureOpenAIInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.chatgpt.ChatGPTInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.azure_chatgpt.AzureChatGPTInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.anthropic_claude.AnthropicClaudeInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.cohere.CohereInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.hugging_face.HFLocalInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.hugging_face_inference.HFInferenceEndpointInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.sagemaker_base.SageMakerBaseInvocationLayer'>, <class 'haystack.nodes.prompt.invocation_layer.sag

### google/flan-t5-base

In [25]:
iteration = iteration
model_name = models_list[iteration]
max_length = 3000
system_message = """
Summarize the messages to create a Frequently Asked Questions document.
"""
prompt = PromptTemplate( # https://docs.haystack.deepset.ai/docs/prompt_node#prompttemplates
    prompt='{query}\n\n Messages: {join(documents)} \n\nSummary: '
)
prompt_dict[iteration] = prompt

prompt_node = PromptNode( # https://docs.haystack.deepset.ai/reference/prompt-node-api # https://docs.haystack.deepset.ai/docs/prompt_node#in-a-pipeline
    # model_name, # Default is "google/flan-t5-base"
    api_key=hf_access_token, 
    max_length=max_length, # The maximum number of tokens the generated text output can have,
    default_prompt_template=prompt,
    )
prompt_node_dict[iteration] = prompt_node
system_message_dict[iteration] = system_message


In [27]:

summarize_pipeline = Pipeline()

summarize_pipeline.add_node(component=prompt_node, name="PromptNode", inputs=["Query"])

output = summarize_pipeline.run(query=system_message, documents=saved_document_store.get_all_documents())
output_dict[iteration] = output
output

The prompt has been truncated from 3053 tokens to 0 tokens so that the prompt length and answer length (3000 tokens) fit within the max token limit (1024 tokens). Shorten the prompt to prevent it from being cut off.


Exception: Exception while running node 'PromptNode': HuggingFace Inference returned an error.
Status code: 400
Response body: {"error":["Error in `parameters.max_new_tokens`: ensure this value is less than or equal to 250"]}
Enable debug logging to see the data that was passed when the pipeline failed.

### philschmid/flan-t5-base-samsum

In [35]:
models_list

['sshleifer/distilbart-cnn-12-6',
 'facebook/bart-large-cnn',
 'philschmid/bart-large-cnn-samsum',
 'philschmid/flan-t5-base-samsum']

In [37]:
iteration = iteration
model_name = models_list[3]
max_length = 3000
system_message = """
Summarize the messages to create a Frequently Asked Questions document.
"""
prompt = PromptTemplate( # https://docs.haystack.deepset.ai/docs/prompt_node#prompttemplates
    prompt='{query}\n\n Messages: {join(documents)} \n\nSummary: '
)
prompt_dict[iteration] = prompt

prompt_node = PromptNode( # https://docs.haystack.deepset.ai/reference/prompt-node-api # https://docs.haystack.deepset.ai/docs/prompt_node#in-a-pipeline
    model_name, # Default is "google/flan-t5-base"
    api_key=hf_access_token, 
    max_length=max_length, # The maximum number of tokens the generated text output can have,
    default_prompt_template=prompt,
    )
prompt_node_dict[iteration] = prompt_node
system_message_dict[iteration] = system_message


In [38]:

summarize_pipeline = Pipeline()

summarize_pipeline.add_node(component=prompt_node, name="PromptNode", inputs=["Query"])

output = summarize_pipeline.run(query=system_message, documents=saved_document_store.get_all_documents())
output_dict[iteration] = output
output

Token indices sequence length is longer than the specified maximum sequence length for this model (3053 > 1024). Running this sequence through the model will result in indexing errors
The prompt has been truncated from 3053 tokens to 0 tokens so that the prompt length and answer length (3000 tokens) fit within the max token limit (1024 tokens). Shorten the prompt to prevent it from being cut off.


Exception: Exception while running node 'PromptNode': HuggingFace Inference returned an error.
Status code: 400
Response body: {"error":["Error in `parameters.max_new_tokens`: ensure this value is less than or equal to 250"]}
Enable debug logging to see the data that was passed when the pipeline failed.

### Try again with fewer tokens

In [39]:
iteration = iteration
model_name = models_list[3]
max_length = 250
system_message = """
Summarize the messages to create a Frequently Asked Questions document.
"""
prompt = PromptTemplate( # https://docs.haystack.deepset.ai/docs/prompt_node#prompttemplates
    prompt='{query}\n\n Messages: {join(documents)} \n\nSummary: '
)
prompt_dict[iteration] = prompt

prompt_node = PromptNode( # https://docs.haystack.deepset.ai/reference/prompt-node-api # https://docs.haystack.deepset.ai/docs/prompt_node#in-a-pipeline
    model_name, # Default is "google/flan-t5-base"
    api_key=hf_access_token, 
    max_length=max_length, # The maximum number of tokens the generated text output can have,
    default_prompt_template=prompt,
    )
prompt_node_dict[iteration] = prompt_node
system_message_dict[iteration] = system_message


In [40]:

summarize_pipeline = Pipeline()

summarize_pipeline.add_node(component=prompt_node, name="PromptNode", inputs=["Query"])

output = summarize_pipeline.run(query=system_message, documents=saved_document_store.get_all_documents())
output_dict[iteration] = output
output

Token indices sequence length is longer than the specified maximum sequence length for this model (3053 > 1024). Running this sequence through the model will result in indexing errors
The prompt has been truncated from 3053 tokens to 774 tokens so that the prompt length and answer length (250 tokens) fit within the max token limit (1024 tokens). Shorten the prompt to prevent it from being cut off.


Exception: Exception while running node 'PromptNode': HuggingFace Inference returned an error.
Status code: 400
Response body: {"error":"The following `model_kwargs` are not used by the model: ['return_full_text', 'stop', 'watermark', 'details'] (note: typos in the generate arguments will also show up in this list)","warnings":["There was an inference error: The following `model_kwargs` are not used by the model: ['return_full_text', 'stop', 'watermark', 'details'] (note: typos in the generate arguments will also show up in this list)","Token indices sequence length is longer than the specified maximum sequence length for this model (775 > 512). Running this sequence through the model will result in indexing errors"]}
Enable debug logging to see the data that was passed when the pipeline failed.

# Try using API request directly to hugging face

## philschmid/flan-t5-base-samsum

In [31]:
import requests

API_URL = "https://api-inference.huggingface.co/models/philschmid/flan-t5-base-samsum"
headers = {"Authorization": f"Bearer {hf_access_token}"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
output = query({
	"inputs": "what is kinesiology?",
})
output

{'error': 'Model philschmid/flan-t5-base-samsum is currently loading',
 'estimated_time': 39.61640548706055}

In [32]:
hf_response_dict = dict()

In [33]:
models_list

['sshleifer/distilbart-cnn-12-6', 'facebook/bart-large-cnn']

In [41]:
import requests

def query_hugging_face(model_name, input):

    API_URL = f"https://api-inference.huggingface.co/models/{model_name}"
    headers = {"Authorization": f"Bearer {hf_access_token}"}
    payload = {"inputs": input}
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()
API_URL = "https://api-inference.huggingface.co/models/philschmid/flan-t5-base-samsum"
headers = {"Authorization": f"Bearer {hf_access_token}"}
	
model = 2
input = "what is kinesiology?"
hf_response_dict[models_list[model]] = query_hugging_face(models_list[model], input)
hf_response_dict[models_list[model]]

{'error': 'Model philschmid/bart-large-cnn-samsum is currently loading',
 'estimated_time': 130.04541015625}

In [43]:
model = 'mistralai/Mistral-7B-Instruct-v0.1'
input = "what is kinesiology?"
hf_response_dict[model] = query_hugging_face(model, input)
hf_response_dict[model]

[{'generated_text': 'what is kinesiology?\nA: Kinesiology is the scientific study of movement and its effects on the human body'}]

# Results

In [ ]:
print(len(output['results']))
print(output['results'][0])

# Look at Discord Messages

## Load raw JSON

In [45]:
import json
def load_json(filename, filepath):
    """
    Load a JSON file using specified file path copied from windows file explorer.
    Back slashes in file path will be converted to forward slashes.

    Arguments:
    - filepath (raw string): Use the format r'<path>'.
    - filename (string).
    """
    filename = f'{filepath}/'.replace('\\','/')+filename
    with open(filename) as file:
        return json.load(file)

filepath = '../data/testing_2023-11-23/'
filename = 'Discord_all_messages.json'
messages = load_json(filename, filepath)
messages

[{'id': 1177344383465558227,
  'author_id': 858179240812609546,
  'content': '',
  'reference_id': None,
  'created_at': '11/23/2023T20:26:15',
  'channel_id': 1106593686223069309,
  'channel_name': 'general'},
 {'id': 1177252923101679626,
  'author_id': 846765301852274718,
  'content': "when embedded with FastAPI is there a different port that Solara loads or it's the same port as uvicorn?",
  'reference_id': 1177252579374268456,
  'created_at': '11/23/2023T14:22:49',
  'channel_id': 1106593686223069309,
  'channel_name': 'general'},
 {'id': 1177252579374268456,
  'author_id': 518050074501185537,
  'content': 'yeah, must we websocket related then',
  'reference_id': 1177252060656304239,
  'created_at': '11/23/2023T14:21:27',
  'channel_id': 1106593686223069309,
  'channel_name': 'general'},
 {'id': 1177252060656304239,
  'author_id': 846765301852274718,
  'content': "I use Chrome and deployed the image on openshift, the Swagger UI for FAST API loads without issues but Solara doesn't l

In [54]:
print(type(messages[0]))
len(messages)

<class 'dict'>


2249

In [48]:
def save_to_json(obj, filename=None, description='Discord_messages_json', append_version=1,
    path='../data'
    ):
    """
    Save Python object as a JSON file.
    Parameters:
    - obj: Python object to be saved.
    - filename: Root of the filename.
    - path (raw string): Use the format r'<path>'. If None, file is saved in same directory as script.
    - append_version (bool): If true, append date and time to end of filename.
    """
    if description:
        filename = f'{description}_{datetime.now().strftime("%Y-%m-%d_%H%M")}'
        append_version = False
    elif filename == None:
        filename = f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_outputs'
        append_version = False
    if path:
        path = f'{path}/'.replace('\\','/')
    if append_version:
        filename += f'_{datetime.now().strftime("%Y-%m-%d_%H%M%S")}'
    filename += '.json'
    with open(path+filename, 'w') as f:
        json.dump(obj, f)
    print(f'Object saved as JSON: {filename}')

save_to_json(messages[0], path=filepath)

Object saved as JSON: Discord_messages_json_2023-11-23_1802.json


## Convert list to dataframe

In [50]:
import pandas as pd
pd.DataFrame(messages)

,id,author_id,content,reference_id,created_at,channel_id,channel_name,channel_parent_id
0,1177344383465558227,858179240812609546,,NaN,11/23/2023T20:26:15,1106593686223069309,general,NaN
1,1177252923101679626,846765301852274718,when embedded with FastAPI is there a differen...,1.177253e+18,11/23/2023T14:22:49,1106593686223069309,general,NaN
2,1177252579374268456,518050074501185537,"yeah, must we websocket related then",1.177252e+18,11/23/2023T14:21:27,1106593686223069309,general,NaN
3,1177252060656304239,846765301852274718,I use Chrome and deployed the image on openshi...,1.177252e+18,11/23/2023T14:19:23,1106593686223069309,general,NaN
4,1177251840035917946,518050074501185537,"that shouldn't happen, make sure you use the s...",1.177252e+18,11/23/2023T14:18:30,1106593686223069309,general,NaN
...,...,...,...,...,...,...,...,...
2244,1177254138799722517,518050074501185537,no further details in the error?,NaN,11/23/2023T14:27:39,1177252923101679626,when embedded with FastAPI is there a,1.106594e+18
2245,1177254082658971658,518050074501185537,even without a reverse proxy?,NaN,11/23/2023T14:27:25,1177252923101679626,when embedded with FastAPI is there a,1.106594e+18
2246,1177253999737569351,846765301852274718,"Yes, on Console it throws error as WebSocket C...",1.177253e+18,11/23/2023T14:27:05,1177252923101679626,when embedded with FastAPI is there a,1.106594e+18
2247,1177253242896388176,518050074501185537,"It's all on the same port, do you have an erro...",NaN,11/23/2023T14:24:05,1177252923101679626,when embedded with FastAPI is there a,1.106594e+18


# *End of Page*